In [7]:
import tensorflow as tf
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential,Model,initializers,layers,Input
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [2]:
import tensorflow as tf
import numpy as np

(x_train,y_train),(x_test,y_test)=tf.keras.datasets.cifar10.load_data()

# Normalization
x_train=x_train/255.

x_test=x_test/255.

x_train.shape

x_train = tf.image.resize(x_train, [224,224]) 

x_test = tf.image.resize(x_test, [224,224]) 

y_train = tf.keras.utils.to_categorical(np.array(y_train))
y_test = tf.keras.utils.to_categorical(np.array(y_test))

In [8]:
dir=r'D:\dataset\butterfly\train'
train_ds=tf.keras.preprocessing.image_dataset_from_directory(
dir,
labels="inferred",
label_mode="int",
class_names=None,
color_mode="rgb",
batch_size=4,
image_size=(224, 224),
shuffle=True,
seed=10,
validation_split=0.1,
subset='training',
interpolation="gaussian",
follow_links=False,
crop_to_aspect_ratio=False,)

validation_ds=tf.keras.preprocessing.image_dataset_from_directory(
dir,
validation_split=0.1,
subset='validation',
shuffle=True,
seed=10,
image_size=(224,224),
batch_size=4)

normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255.)

normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]

Found 1257 files belonging to 10 classes.
Using 1132 files for training.
Found 1257 files belonging to 10 classes.
Using 125 files for validation.


In [9]:
dir=r'D:\dataset\butterfly\train'
train_ds=tf.keras.preprocessing.image_dataset_from_directory(
dir,
labels="inferred",
label_mode="int",
class_names=None,
color_mode="rgb",
batch_size=4,
image_size=(224, 224),
shuffle=True,
seed=10,
validation_split=0.1,
subset='training',
interpolation="gaussian",
follow_links=False,
crop_to_aspect_ratio=False,)

validation_ds=tf.keras.preprocessing.image_dataset_from_directory(
dir,
validation_split=0.1,
subset='validation',
shuffle=True,
seed=10,
image_size=(224, 224),
batch_size=4)

normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255.)

normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]

Found 1257 files belonging to 10 classes.
Using 1132 files for training.
Found 1257 files belonging to 10 classes.
Using 125 files for validation.


In [10]:
model=Sequential([
    layers.Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),strides=(1,1),padding='same'),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(kernel_size=(3,3),filters=64,strides=(1,1),padding='same'),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.MaxPooling2D(pool_size=(2,2),strides=(2,2)),

    layers.Conv2D(filters=128,kernel_size=(3,3),strides=(1,1),padding='same'),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(filters=128,kernel_size=(3,3),strides=(1,1),padding='same'),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(filters=128,kernel_size=(3,3),strides=(1,1),padding='same'),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.MaxPooling2D(pool_size=(2,2),strides=(2,2)),
 
    layers.Conv2D(filters=256,kernel_size=(3,3),strides=(1,1),padding='same'),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(filters=256,kernel_size=(3,3),strides=(1,1),padding='same'),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Conv2D(filters=256,kernel_size=(3,3),strides=(1,1),padding='same'),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.MaxPooling2D(pool_size=(2,2),strides=(2,2)),

    layers.Flatten(),
    layers.Dense(units=1024),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Dense(units=1024),
    layers.BatchNormalization(),
    layers.ReLU(),
    layers.Dense(units=10,activation='softmax')
])

In [ ]:
plot_model(model,to_file='VGGnet_16.png',show_shapes=True)

In [10]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 224, 224, 64)      1792      
                                                                 
 batch_normalization_18 (Bat  (None, 224, 224, 64)     256       
 chNormalization)                                                
                                                                 
 re_lu_18 (ReLU)             (None, 224, 224, 64)      0         
                                                                 
 conv2d_19 (Conv2D)          (None, 224, 224, 64)      36928     
                                                                 
 batch_normalization_19 (Bat  (None, 224, 224, 64)     256       
 chNormalization)                                                
                                                                 
 re_lu_19 (ReLU)             (None, 224, 224, 64)     

In [12]:
len(model.layers)

31

In [11]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy' ,metrics=['acc'])

In [12]:
history=model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=1,batch_size=8,verbose=1)

NameError: name 'x_train' is not defined

In [5]:
with tf.device("/device:GPU:0"):
    history=model.fit(train_ds,validation_data=validation_ds,epochs=10,batch_size=4,verbose=1)

Epoch 1/10
283/283 [==============================] - 1111s 4s/step - loss: 38.4654 - acc: 0.2120 - val_loss: 12.9152 - val_acc: 0.3200
Epoch 2/10
 46/283 [===>..........................] - ETA: 15:05 - loss: 3.8342 - acc: 0.2011

KeyboardInterrupt: 

In [ ]:
model.load_weights(model_checkpoint_best_path)
test_loss,test_acc,test_presicion,test_recall=model.evaluate(x=x_test,y=y_test,verbose=1)